# Exploring 'Hacker News' Posts - “What time should you post to receive the highest average of comments?”

In this project, we'll work a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

For convenience of this practice, the data has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling the remaining submissions.

The full data set is available right [here](https://www.kaggle.com/hacker-news/hacker-news-posts).

In our analysis we are specifically interested in posts that begin with 'Ask HN'(To ask the hacker news community about specific topics) and 'Show HN'(to show the hacker news community their projects, product or something interesting).

So we will compare these two type **Ask HN** and **Show HN** to determine which one gets more reach or comments and whether the most comments are posted during a specific time of day.

In [1]:
from csv import reader 
import datetime as dt

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

headers = hn[:1]
hn = hn[1:]

In [2]:
#Displaying header and first 5 rows of the dataset
print(headers,'\n')

for row in hn[:5]:
    print(row)
    print('\n')

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']] 

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




### Extracting Ask HN and Show HN posts

To find the posts that begin with either `Ask HN` or `Show HN`, we'll use the string method `startswith`. Since, `startswith` is case sensitive, here in this project we are using `lower` method, to have a control on case. 

In [3]:
ask_posts = [] # all posts relevant to Ask HN
show_posts = []# all posts relevant to Show HN
other_posts = []# other posts

for row in hn:
    title = row[1]
    
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
        
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print('Number of all posts: ', len(hn),'\n')
print('Number of ask posts: ', len(ask_posts))
print('Number of show posts: ', len(show_posts))
print('Number of other posts: ', len(other_posts))


Number of all posts:  20100 

Number of ask posts:  1744
Number of show posts:  1162
Number of other posts:  17194


To verify that the list works fine, we have to take a look at the first and last two rows of each list.

In [4]:
for entry in range(0,2):
    print(ask_posts[entry][1])
    print(ask_posts[-entry-1][1])

print('\n')

for entry in range(0,2):
    print(show_posts[entry][1])
    print(show_posts[-entry-1][1])

Ask HN: How to improve my personal website?
Ask HN: Why are papers still published as PDFs?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: How do you balance a serious relationship with starting a company?


Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform
Show HN: Parse recipe ingredients using JavaScript
Show HN: Something pointless I made
Show HN: PhantomJsCloud, Headless Browser SaaS


### Calculating the average number of comments for Ask HN and Show HN posts.

In [5]:
#find total number for the ask_posts category
total_ask_comments = 0

for posts in ask_posts:
    comments = int(posts[4])
    total_ask_comments += comments

#calculate the average number of comments in ask posts
avg_ask_comments = total_ask_comments / len(ask_posts)
print('Average ask posts comments: ','{:.2f}'.format(avg_ask_comments))

Average ask posts comments:  14.04


In [6]:
#find total number for the show_posts category
total_show_comments = 0

for posts in show_posts:
    comments = int(posts[4])
    total_show_comments += comments

#calculate the average number of comments in show posts
avg_show_comments = total_show_comments / len(show_posts)

print('Average show posts comments: ','{:.2f}'.format(avg_show_comments))

Average show posts comments:  10.32


In [7]:
#to compare the average of ask and int


comments_diff = abs(avg_show_comments - avg_ask_comments)

if avg_ask_comments > avg_show_comments:
    print('The ask sections receives ' + '{:.2f}'.format(comments_diff) + ' more comments.')
elif avg_show_comments > avg_ask_comments:
    print('The show sections receives ' + '{:.2f}'.format(comments_diff) + ' more comments.')
else:
    print('Both the sections receives equal number of posts on average.')

The ask sections receives 3.72 more comments.


### Finding the number of Ask posts and comments by hour created.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.


In [49]:
result_list = []
# we iterate trough each post in ask_post, then asign
# the timestamp to created_at and the number of comments
# to num_comments. At the end we append a list with that
# data to the result_list

for posts in ask_posts:
    dates = posts[-1]
    n_comm = int(posts[4])
    result_list.append([dates, n_comm])

counts_by_hour = {} # contains the number of ask posts created during each hour of the day
comments_by_hour = {} # contains the corresponding number of comments ask posts created at each hour received

for row in result_list:
    date_str = row[0]
    num_comments = row[1]
    dt_object = dt.datetime.strptime(date_str, '%m/%d/%Y %H:%M') # we convert the date string to a datetime object
    hour = dt_object.strftime('%H') # we ectract the hour (%H) from the datetime object
    
    # now we create a frequent table and count the hour to get
    # the number of posts for each hour and set the comments by hour
    # equal to the comment number to get the comments in each hour.
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments
        

In [43]:
#with sorted(count_by_hour) we sort each hour from 00 to 23
for hour in sorted(counts_by_hour):
    print('Hour: ', hour,'Posts: ', counts_by_hour[hour])

Hour:  00 Posts:  55
Hour:  01 Posts:  60
Hour:  02 Posts:  58
Hour:  03 Posts:  54
Hour:  04 Posts:  47
Hour:  05 Posts:  46
Hour:  06 Posts:  44
Hour:  07 Posts:  34
Hour:  08 Posts:  48
Hour:  09 Posts:  45
Hour:  10 Posts:  59
Hour:  11 Posts:  58
Hour:  12 Posts:  73
Hour:  13 Posts:  85
Hour:  14 Posts:  107
Hour:  15 Posts:  116
Hour:  16 Posts:  108
Hour:  17 Posts:  100
Hour:  18 Posts:  109
Hour:  19 Posts:  110
Hour:  20 Posts:  80
Hour:  21 Posts:  109
Hour:  22 Posts:  71
Hour:  23 Posts:  68


In [57]:
#To get the hour with most posts
#we iterate through counts_by_hour and
#if a value is greater than max_by_hour 
#then we save it to max_hour

max_by_hour = 0
max_hour = []

for row in counts_by_hour:
    if counts_by_hour[row] > max_by_hour:
        max_by_hour = counts_by_hour[row]
        max_hour = [row, counts_by_hour[row]]

print('By ',max_hour[1], ' posts, ',  max_hour[0], '\'o clock has the highest number of posts.')

By  116  posts,  15 'o clock has the highest number of posts.


In [58]:
#with sorted(comments_by_hour) we sort each hour from 00 to 23
for hour in sorted(comments_by_hour):
    print('Hour: ', hour,'Comments: ', comments_by_hour[hour])

Hour:  00 Comments:  447
Hour:  01 Comments:  683
Hour:  02 Comments:  1381
Hour:  03 Comments:  421
Hour:  04 Comments:  337
Hour:  05 Comments:  464
Hour:  06 Comments:  397
Hour:  07 Comments:  267
Hour:  08 Comments:  492
Hour:  09 Comments:  251
Hour:  10 Comments:  793
Hour:  11 Comments:  641
Hour:  12 Comments:  687
Hour:  13 Comments:  1253
Hour:  14 Comments:  1416
Hour:  15 Comments:  4477
Hour:  16 Comments:  1814
Hour:  17 Comments:  1146
Hour:  18 Comments:  1439
Hour:  19 Comments:  1188
Hour:  20 Comments:  1722
Hour:  21 Comments:  1745
Hour:  22 Comments:  479
Hour:  23 Comments:  543


In [61]:
#To get the hour with most comments
#we iterate through comments_by_hour and
#if a value is greater than max_by_hour 
#then we save it to max_hour

max_by_comments = 0
max_comments = []

for row in comments_by_hour:
    if comments_by_hour[row] > max_by_comments:
        max_by_comments = comments_by_hour[row]
        max_comments = [row, comments_by_hour[row]]

print('By ',max_comments[1], ' comments, ',  max_comments[0], '\'o clock has the highest number of comments.')

By  4477  comments,  15 'o clock has the highest number of comments.


In [62]:
#calculating average number of comments for posts during each hour

avg_by_hour = []

for count in comments_by_hour:
    
    avg_by_hour.append([count, comments_by_hour[count] / counts_by_hour[count]])
    
#with sorted, display the output for better readability

for row in sorted(avg_by_hour):
    print(row)
   

['00', 8.127272727272727]
['01', 11.383333333333333]
['02', 23.810344827586206]
['03', 7.796296296296297]
['04', 7.170212765957447]
['05', 10.08695652173913]
['06', 9.022727272727273]
['07', 7.852941176470588]
['08', 10.25]
['09', 5.5777777777777775]
['10', 13.440677966101696]
['11', 11.051724137931034]
['12', 9.41095890410959]
['13', 14.741176470588234]
['14', 13.233644859813085]
['15', 38.5948275862069]
['16', 16.796296296296298]
['17', 11.46]
['18', 13.20183486238532]
['19', 10.8]
['20', 21.525]
['21', 16.009174311926607]
['22', 6.746478873239437]
['23', 7.985294117647059]


In [78]:
#Let's swap the places of avg_by_hour
#to sort the data at more convenience
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print('Most number of comments where obsevered during:')
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
    
for average, hr in sorted_swap[:5]: 
    hour = dt.datetime.strptime(hr,'%H')#convert the string to a datetime object
    time = hour.strftime('%H:%M')#format the datetime object
    print('{time}: {average:.2f} comments per post'.format(time = time, average = average))
    

Most number of comments where obsevered during:
15:00: 38.59 comments per post
02:00: 23.81 comments per post
20:00: 21.52 comments per post
16:00: 16.80 comments per post
21:00: 16.01 comments per post


The best time to get maximum reach in Hacker News is to create a post at 15:00. The time in this data set is Eastern Standard Time. Therefore, 10.5 hours  should be added to it in order to convert it into Indian Standard Time.

In [80]:
print('Top 5 hours in IST for Ask Posts comments:')

for average, hr in sorted_swap[:5]: 
    hour = dt.datetime.strptime(hr,'%H')
    ist = hour + dt.timedelta(hours = 10.5)
    time = ist.strftime('%H:%M')
    print('{time}: {average:.2f} comments per post'.format(time = time, average = average))
    

Top 5 hours in IST for Ask Posts comments:
01:30: 38.59 comments per post
12:30: 23.81 comments per post
06:30: 21.52 comments per post
02:30: 16.80 comments per post
07:30: 16.01 comments per post


### Calculating the average number of points for Ask HN and Show HN posts.

In [83]:
total_ask_points = 0

for row in ask_posts:
    n_points = row[3]
    total_ask_points += int(n_points)

avg_ask_points = total_ask_points / len(ask_posts)

print('Average number of points per posts in Ask HN are: ','{:.2f}'.format(avg_ask_points))


Average number of points per posts in Ask HN are:  15.06


In [85]:
total_show_points = 0

for row in show_posts:
    n_points = row[3]
    total_show_points += int(n_points)
    
avg_show_points = total_show_points / len(show_posts)

print('Average number of points per posts in Show HN are: ','{:.2f}'.format(avg_show_points))


Average number of points per posts in Show HN are:  27.56


In [89]:
#comparing the averages of both Ask and Show sections

point_diff = abs(avg_ask_points - avg_show_points)

if avg_show_points > avg_ask_points:
    print('The Show section receives ', '{:.2f}'.format(point_diff), 'more points than the other.')
elif avg_ask_points > avg_show_points:
    print('The Ask section receives ', '{:.2f}'.format(point_diff), 'more points than the other.')

The Show section receives  12.49 more points than the other.


### Finding the number of Show posts points by the  hour it was created.

Since show posts are more likely to receive points, we'll focus our remaining analysis just on these posts as per the same steps followed in analysing the number of comments.


In [101]:
result_list = [] #A list of lists which holds relevant columns from the dataset for our analysis

for row in show_posts:
    dates = row[-1]
    n_points = int(row[3])
    result_list.append([dates, n_points])
    
posts_by_hour = {}
points_by_hour = {}

for row in result_list:
    hr = row[0]
    n_points = row[1]
    time = dt.datetime.strptime(hr,'%m/%d/%Y %H:%M')
    hr = time.strftime('%H')
    
    if hr not in posts_by_hour:
        posts_by_hour[hr] = 1
        points_by_hour[hr] = 1
    else:
        posts_by_hour[hr] += 1
        points_by_hour[hr] += n_points
        

In [102]:
for hour in sorted(posts_by_hour):
    print('Hour: ',hour,' Posts: ', posts_by_hour[hour])

Hour:  00  Posts:  31
Hour:  01  Posts:  28
Hour:  02  Posts:  30
Hour:  03  Posts:  27
Hour:  04  Posts:  26
Hour:  05  Posts:  19
Hour:  06  Posts:  16
Hour:  07  Posts:  26
Hour:  08  Posts:  34
Hour:  09  Posts:  30
Hour:  10  Posts:  36
Hour:  11  Posts:  44
Hour:  12  Posts:  61
Hour:  13  Posts:  99
Hour:  14  Posts:  86
Hour:  15  Posts:  78
Hour:  16  Posts:  93
Hour:  17  Posts:  93
Hour:  18  Posts:  61
Hour:  19  Posts:  55
Hour:  20  Posts:  60
Hour:  21  Posts:  47
Hour:  22  Posts:  46
Hour:  23  Posts:  36


In [110]:
max_by_hour = 0
max_posts = []

for hour in posts_by_hour:
    if posts_by_hour[hour] > max_by_hour:
        max_by_hour = posts_by_hour[hour]
        max_posts = [hour, posts_by_hour[hour]]
        
print('By',max_posts[1] ,'posts,', max_posts[0],'\'o clock has the highest number of posts.')

By 99 posts, 13 'o clock has the highest number of posts.


In [114]:
for hour in sorted(points_by_hour):
    print('Hour: ',hour,' Points: ', points_by_hour[hour])
    
max_by_hour = 0
max_posts = []

for hour in points_by_hour:
    if points_by_hour[hour] > max_by_hour:
        max_by_hour = points_by_hour[hour]
        max_posts = [hour, points_by_hour[hour]]
        
print('By',max_posts[1] ,'posts,', max_posts[0],'\'o clock has the highest number of posts.')

Hour:  00  Points:  1165
Hour:  01  Points:  692
Hour:  02  Points:  337
Hour:  03  Points:  672
Hour:  04  Points:  374
Hour:  05  Points:  102
Hour:  06  Points:  372
Hour:  07  Points:  492
Hour:  08  Points:  515
Hour:  09  Points:  545
Hour:  10  Points:  680
Hour:  11  Points:  1479
Hour:  12  Points:  2537
Hour:  13  Points:  2438
Hour:  14  Points:  2162
Hour:  15  Points:  2186
Hour:  16  Points:  2627
Hour:  17  Points:  2517
Hour:  18  Points:  2215
Hour:  19  Points:  1697
Hour:  20  Points:  1819
Hour:  21  Points:  862
Hour:  22  Points:  1110
Hour:  23  Points:  1518
By 2627 posts, 16 'o clock has the highest number of posts.


In [126]:
avg_by_hour = []

for hour in posts_by_hour:
    avg_by_hour.append([points_by_hour[hour] / posts_by_hour[hour], hour])
    
sorted_avg_by_hour = sorted(avg_by_hour, reverse = True)
print('Top 5 time to post in Show HN in IST:')
for average, hr in sorted_avg_by_hour[:5]:
    hour = dt.datetime.strptime(hr, '%H')
    ist = hour + dt.timedelta(hours = 10.3)
    hr = ist.strftime('%H:%M')
    print('{time}: {average:.2f} points per posts'.format(time=hr, average = average))

Top 5 time to post in Show HN in IST:
09:18: 42.17 points per posts
22:18: 41.59 points per posts
10:18: 37.58 points per posts
04:18: 36.31 points per posts
21:18: 33.61 points per posts


In our analysis we found that most number of comments are posted in Ask HN section whereas the most number of points are awarded in Show HN section. So it is clear that most number of comments will be going to Ask HN as it is the best way to clear doubts or queries. Show HN received lesser number of comments but a higher number of points.


In order for our articles to receives higher number of comments it is ideal to make that post between **12:30 am** and **1:30 am** IST. 

The time period seems to vary in case of Show HN section as this section will be more dependant on the content. But, in order to receive high responses it's better to post during the day time rather than during night. Another interesting fact observed is that most points were given during a an hour in the morning and an hour in the night. Which means that people tend to skim through the show section in the morning and before going to bed. So a quality post during this time will attact more viewers.

-

Even though this analysis has now given us an estimate of when it is worth writing an article, it doesn't mean that we will always get the most comments if we publish at that time. So, of course, the content of the article still plays a key role.